In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [2]:
example = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\SLAT\memory_disorder_SLAT\regex_dataset.xlsx", sheet_name = "Sheet1", index_col = 0)

In [3]:
example.head()

,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
PatientID,,,,,,,,,,
Z6708893,1.004451e+10,100100942.0,3.115772e+09,Progress Notes,1000021019,955432079,2016-04-28,----------------------------------------------...,"""Visit Other Routine general medical examinati...","[""memory change""]"
Z7766129,1.002544e+10,100240713.0,3.116814e+09,H&P,1000431290,954962537,2016-04-28,----------------------------------------------...,"""\u2022 Doxepin Other (See Comments) vivid dre...","[""memory problem""]"
Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""accident moving next year due to extra bedroo...","[""dementia""]"
Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""techniques discuss again correlations long te...","[""dementia""]"
Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""benzo rxs 4/98, 9/97, 5/04 Family history: -m...","[""dementia""]"


# Padded Matches

In [74]:
df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Padded_Matches\apoe_1_padded_matches.csv")
df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\Padded_Matches\apoe_2_padded_matches.csv")
df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\Padded_Matches\apoe_3_padded_matches.csv")
df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\Padded_Matches\apoe_4_padded_matches.csv")
df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\Padded_Matches\apoe_5_padded_matches.csv")

In [ ]:
len(df1), len(df2), len(df3), len(df4), len(df5)

In [ ]:
frames = [df1, df2,df3, df4, df5]
df = pd.concat(frames)

In [35]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC', 'buckets',
       'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location', 'regex_sent',
       'regex_match', 'match_count', 'pruned_regex_location',
       'merged_row_location', 'merged_regex_sent', 'merged_regex_match',
       'sequence_level_regex_location', 'sequence_level_regex_match',
       'char_count', 'padded_merged_row_location', 'padded_merged_regex_sent',
       'padded_char_count', 'note_char_count'],
      dtype='object')

In [36]:
example.columns

Index(['MRN', 'EMPI', 'PatientEncounterID', 'InpatientNoteTypeDSC', 'NoteID',
       'NoteCSNID', 'ContactDTS', 'NoteTXT', 'regex_sent', 'regex_match'],
      dtype='object')

In [37]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'EncounterTypeDSC'])
df.head()

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,...,merged_row_location,merged_regex_sent,merged_regex_match,sequence_level_regex_location,sequence_level_regex_match,char_count,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,471433064,Progress Notes,1,4.532520e+08,1.0,...,"[(5750, 5958)]",------- ncer. She passed secondary to this. He...,[' cerebral'],"[(108, 116)]",['cerebral'],213,"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,NaN
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,513142600,Progress Notes,1,4.924833e+08,1.0,...,"[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"[(108, 114)]",['memory'],210,"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,NaN
2,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,2370049295,Progress Notes,1,2.338373e+09,1.0,...,"[(8641, 8849)]","------- ractures, obesity, insulin resistance,...",[' dementia'],"[(108, 116)]",['dementia'],208,"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,NaN
3,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,4959788710,Progress Notes,1,4.951007e+09,1.0,...,"[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(321, 330), (549, 558), (539, 545), (108, 112...","['cognition', 'memory', 'moca']",764,"[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,NaN
4,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,4570521668,Progress Notes,1,4.558686e+09,1.0,...,"[(5367, 5576), (5593, 5812)]",------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(108, 117), (336, 345), (326, 332)]","['cognition', 'memory']",441,"[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,NaN


## Computing Matches For Padded Sequences

In [38]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [39]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[re.compile('\\bdementia\\b', re.IGNORECASE), re.compile('\\bcognition\\b', re.IGNORECASE), re.compile('\\bmemory\\b', re.IGNORECASE), re.compile('\\bMMSE\\b', re.IGNORECASE), re.compile('\\bMOCA\\b', re.IGNORECASE), re.compile('\\bAlzheimer\\b', re.IGNORECASE), re.compile('\\bcognitive\\s*impairment\\b', re.IGNORECASE), re.compile('(?-i:)\\bMCI\\b'), re.compile('\\bcerebellar\\b', re.IGNORECASE), re.compile('\\bNeurocognitive\\b', re.IGNORECASE), re.compile('\\blewy\\b', re.IGNORECASE), re.compile("\\bpick's\\b", re.IGNORECASE), re.compile('\\bCorticobasal\\b', re.IGNORECASE), re.compile('\\bcerebral\\b', re.IGNORECASE), re.compile('\\bcerebrovascular\\b', re.IGNORECASE), re.compile('\\bamnesia\\b', re.IGNORECASE), re.compile('(?-i:)\\bAD\\b'), re.compile('(?-i:)\\bLBD\\b')]


In [40]:
l = []
for note in tqdm(df["padded_merged_regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append(str(curr))

df["padded_regex_match"] = l

100%|████████████████████████████████████████████████████████████████████████| 239629/239629 [02:43<00:00, 1465.92it/s]


In [41]:
df.head()

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,...,merged_regex_sent,merged_regex_match,sequence_level_regex_location,sequence_level_regex_match,char_count,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,padded_regex_match
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,471433064,Progress Notes,1,4.532520e+08,1.0,...,------- ncer. She passed secondary to this. He...,[' cerebral'],"[(108, 116)]",['cerebral'],213,"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,NaN,['cerebral']
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,513142600,Progress Notes,1,4.924833e+08,1.0,...,------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"[(108, 114)]",['memory'],210,"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,NaN,['memory']
2,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,2370049295,Progress Notes,1,2.338373e+09,1.0,...,"------- ractures, obesity, insulin resistance,...",[' dementia'],"[(108, 116)]",['dementia'],208,"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,NaN,['dementia']
3,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,4959788710,Progress Notes,1,4.951007e+09,1.0,...,------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(321, 330), (549, 558), (539, 545), (108, 112...","['cognition', 'memory', 'moca']",764,"[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,NaN,"['cognition', 'memory', 'moca']"
4,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,4570521668,Progress Notes,1,4.558686e+09,1.0,...,------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(108, 117), (336, 345), (326, 332)]","['cognition', 'memory']",441,"[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,NaN,"['cognition', 'memory']"


In [42]:
df.columns

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'buckets', 'NoteCSNID', 'LineNBR',
       'NoteTXT', 'regex_location', 'regex_sent', 'regex_match', 'match_count',
       'pruned_regex_location', 'merged_row_location', 'merged_regex_sent',
       'merged_regex_match', 'sequence_level_regex_location',
       'sequence_level_regex_match', 'char_count',
       'padded_merged_row_location', 'padded_merged_regex_sent',
       'padded_char_count', 'note_char_count', 'padded_regex_match'],
      dtype='object')

## Dropping Rest of Columns

In [43]:
# Index(['MRN', 'EMPI', 'PatientEncounterID', 'InpatientNoteTypeDSC', 'NoteID',
#        'NoteCSNID', 'ContactDTS', 'NoteTXT', 'regex_sent', 'regex_match'],
#       dtype='object')

In [44]:
df = df.drop(columns = ['buckets', 'LineNBR', 'regex_location', 'regex_sent', 'regex_match', 'match_count','pruned_regex_location', 'merged_row_location', 'merged_regex_sent','merged_regex_match', 'sequence_level_regex_location','sequence_level_regex_match', 'char_count','padded_merged_row_location', 'padded_char_count',])

In [47]:
df = df.drop(columns = ['note_char_count'])

In [48]:
df.columns

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT',
       'padded_merged_regex_sent', 'padded_regex_match'],
      dtype='object')

In [50]:
df.columns = ['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT', 'regex_sent', 'regex_match']

In [51]:
df.head()

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,NoteCSNID,NoteTXT,regex_sent,regex_match
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,471433064,Progress Notes,4.532520e+08,----------------------------------------------...,ked at HMS in the department of psychobiology ...,['cerebral']
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,513142600,Progress Notes,4.924833e+08,----------------------------------------------...,or lymphadenopathy. Chest: Clear to auscultati...,['memory']
2,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,2370049295,Progress Notes,2.338373e+09,----------------------------------------------...,ession alone in the meta-analysis. We discusse...,['dementia']
3,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,4959788710,Progress Notes,4.951007e+09,----------------------------------------------...,------- ing tylenol really accept for this we...,"['cognition', 'memory', 'moca']"
4,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,4570521668,Progress Notes,4.558686e+09,----------------------------------------------...,------- in is related to cancer. Reassuringly...,"['cognition', 'memory']"


In [52]:
df.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", index = False)

In [67]:
df = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", sheet_name = "Sheet1", index_col = 0) 

In [68]:
df['regex_sent'] = df['regex_sent'].apply(json.dumps)
df['regex_match'] = df['regex_match'].apply(json.dumps)

In [69]:
df.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_padded_matches.xlsx", index = False)

# Not-Padded Matches

In [53]:
df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Padded_Matches\apoe_1_not_padded_matches.csv")
df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\Padded_Matches\apoe_2_not_padded_matches.csv")
df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\Padded_Matches\apoe_3_not_padded_matches.csv")
df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\Padded_Matches\apoe_4_not_padded_matches.csv")
df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\Padded_Matches\apoe_5_not_padded_matches.csv")

In [54]:
frames = [df1, df2,df3, df4, df5]
df = pd.concat(frames)

In [55]:
example.columns

Index(['MRN', 'EMPI', 'PatientEncounterID', 'InpatientNoteTypeDSC', 'NoteID',
       'NoteCSNID', 'ContactDTS', 'NoteTXT', 'regex_sent', 'regex_match'],
      dtype='object')

In [57]:
df.columns, len(df)

(Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
        'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
        'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC', 'buckets',
        'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location', 'regex_sent',
        'regex_match', 'match_count', 'pruned_regex_location',
        'merged_row_location', 'merged_regex_sent', 'merged_regex_match',
        'sequence_level_regex_location', 'sequence_level_regex_match',
        'char_count', 'note_char_count'],
       dtype='object'),
 40471)

In [58]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'EncounterTypeDSC'])
df.head()

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,...,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent,merged_regex_match,sequence_level_regex_location,sequence_level_regex_match,char_count,note_char_count
0,100000272,10040002098,Z6352398,3.329631e+09,2020-11-20 00:00:00.0000000,4892065342,Progress Notes,1,4.882759e+09,1.0,...,['cognition'],1,"[(100, 110)]","[(181, 390), (479, 688), (2048, 2269), (6184, ...","------- ial Source of info:? Patient, Chart Mo...","[' cognition', ' memory', ' cognitive impairme...","[(2661, 2669), (108, 117), (326, 335), (989, 9...","['dementia', 'cognition', 'memory', 'moca', 'c...",2760,NaN
1,100001150,10027033512,Z6353136,3.085646e+09,2015-07-14 00:00:00.0000000,519778366,Discharge Summary,1,5.104843e+08,1.0,...,['dementia'],1,"[(100, 109)]","[(1496, 1704), (2465, 2675), (4481, 4687), (54...",------- ory is obtained from patient and pt's ...,"[' dementia', ' memory', ' cerebellar']","[(108, 116), (1348, 1356), (1565, 1573), (544,...","['dementia', 'memory', 'cerebellar']",1869,NaN
2,100001150,10027033512,Z6353136,3.114434e+09,2016-05-19 00:00:00.0000000,1041567131,Progress Notes,1,9.940304e+08,1.0,...,['Dementia'],1,"[(100, 109)]","[(245, 596), (1735, 2042), (2951, 3159)]",------- ed during the visit to clarify patient...,"[' cognition', ' dementia', ' cognitive impair...","[(108, 116), (251, 259), (784, 792), (566, 575...","['dementia', 'cognition', 'cognitive impairment']",886,NaN
3,100001150,10027033512,Z6353136,3.121057e+09,2016-06-07 00:00:00.0000000,1068541875,ED Provider Notes,1,1.019340e+09,1.0,...,['dementia'],1,"[(100, 109)]","[(128, 505), (5897, 6105), (9800, 10008)]",------- Hospital Emergency Department History...,[' dementia'],"[(108, 116), (277, 285), (494, 502)]",['dementia'],813,NaN
4,100001150,10027033512,Z6353136,3.121057e+09,2016-06-07 00:00:00.0000000,1068662944,H&P,1,1.019402e+09,1.0,...,['dementia'],1,"[(100, 109)]","[(258, 466), (903, 1110), (1193, 1401), (5953,...",------- emale who presents with syncope. Ms. B...,"[' amnesia', ' dementia', ' memory']","[(108, 116), (541, 549), (1190, 1198), (758, 7...","['dementia', 'memory', 'amnesia']",1297,NaN


In [59]:
df.columns

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'buckets', 'NoteCSNID', 'LineNBR',
       'NoteTXT', 'regex_location', 'regex_sent', 'regex_match', 'match_count',
       'pruned_regex_location', 'merged_row_location', 'merged_regex_sent',
       'merged_regex_match', 'sequence_level_regex_location',
       'sequence_level_regex_match', 'char_count', 'note_char_count'],
      dtype='object')

In [61]:
df = df.drop(columns = ['buckets', 'LineNBR', 'regex_location', 'regex_sent', 'regex_match', 'match_count','pruned_regex_location', 'merged_row_location', 'merged_regex_sent','merged_regex_match', 'char_count'])

In [62]:
df.columns

Index(['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT',
       'sequence_level_regex_location', 'sequence_level_regex_match',
       'note_char_count'],
      dtype='object')

In [63]:
df = df.drop(columns = ['note_char_count'])

In [64]:
df.columns = ['EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'NoteID', 'InpatientNoteTypeDSC', 'NoteCSNID', 'NoteTXT',
       'regex_sent', 'regex_match']

In [65]:
df['regex_sent'] = df['regex_sent'].apply(json.dumps)
df['regex_match'] = df['regex_match'].apply(json.dumps)

In [66]:
df.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\apoe_not_padded_matches.xlsx", index = False)